In [2]:
import pandas as pd
import numpy as np
import plotnine as pn
from matplotlib import pyplot as plt
import plotly.express as px

In [3]:
tow = pd.read_excel('tow.xlsx')
tow.columns = map(str.lower, tow.columns)
tow.columns = tow.columns.str.replace(' ', '_')
tow.columns = tow.columns.str.replace('?', '')
faction_replace = {'Warriors of Chaos': 'W_Chaos', 'Tomb Kings of Khemri': 'Tomb Kings', 'Kingdom of Bretonnia': 'Bretonnia', 'Ogre Kingdoms': 'Ogres', 'Wood Elf Realms': 'Wood Elves', 'Dwarfen Mountain Holds': 'Dwarves', 'Orc and Goblin Tribes': 'O&G', 'Empire of Man': 'Empire', 'Beastmen Breyherds': 'Beastmen', 'High Elf Realms': 'High Elves', 'Daemons of Chaos': 'D_Chaos'}
result_replace = {1:0.5, 2:1}
tow.replace({'player1_faction':faction_replace}, inplace=True)
tow.replace({'player2_faction':faction_replace}, inplace=True)
tow.replace({'player1_result': result_replace}, inplace=True)
tow.replace({'player2_result': result_replace}, inplace=True)

# Warhammer: The Old World
## A Statistical Analysis of Tournament Win Rates
### By: Jacob M. Lundeen

I was first introduced to Games Workshop's (GW) Warhammer: Fantasy Battles (WFB) back in the mid 1990's in Fairfax, Virginia. I was in the fifth or sixth grade and wondered by a GW store with my parents. I convinced them to take me in and managed to walk out with my first eight pack of skeleton warriors (Undead for life!). Over the next couple of years I would acquire a healthy collection of miniatures that was significantly boosted by my winning a White Dwarf survey contest and bringing home $5k worth of Tyranids, Dogs of War, Blood Bowl, and a others. As I grew up, I lost interest in WFB for various reasons as a lot of kids do. However, with WFB being reborn as Warhammer: The Old World (TOW) earlier this year, and me as an adult able to afford this hobby, I jumped back in and haven't looked back.

Since I've rediscovered TOW, I've wanted to combine TOW with one of my other passions, Data Science. With TOW only having returned earlier this year, the available data set is still immature. As we'll see there are many instances of incredible small sample sizes when we look at specific faction versus faction. But my hope is this will give an interesting look into where the game currently stands, where it might need to go, and lead to other interesting questions to ask and explore.

## The Data
The data I am using comes from the Woehammer website (woehammer.com). They compiled their dataset by scraping several tournament tracking websites such as Best Coast Pairings (bestcoastpairings.com). Tournaments use these websites to manage and track their results. I've already done the intial cleaning of the dataset, getting the column names correct and replacing the faction names with shorter names for ease of reading.

In [4]:
tow.shape

(2030, 31)

In [5]:
tow.head()

,pairing_id,round,player1_name,player2_name,player1_result,player2_result,player1_score,player2_score,event_name,event_date,...,points,rounds,ruleset,full_data,mirror_match,players,legacy_lists,allies,rule_of_3,faction_v_faction
0,405136,1,Ridvan Martinez,David Clarke,1.0,0.0,1770.0,901.0,The First Old World Forgemaster,2024-02-03 14:00:00,...,2000,3,v1.1,N,N,12,Y,NaN,NaN,Wood Elf Realms
1,405155,1,Mike Summerfeldt,Russ Jeffery,0.0,1.0,450.0,1920.0,The First Old World Forgemaster,2024-02-03 14:00:00,...,2000,3,v1.1,Y,N,12,Y,NaN,NaN,Warriors of ChaosWood Elf Realms
2,405171,1,Brandon Deamer,Eddie Crampton,1.0,0.0,1745.0,1531.0,The First Old World Forgemaster,2024-02-03 14:00:00,...,2000,3,v1.1,Y,N,12,Y,NaN,NaN,Tomb Kings of KhemriHigh Elf Realms
3,405192,1,taylor hanson,Luka Pavicevic,0.0,1.0,867.0,1205.0,The First Old World Forgemaster,2024-02-03 14:00:00,...,2000,3,v1.1,Y,N,12,Y,NaN,NaN,Kingdom of BretonniaDwarfen Mountain Holds
4,405199,1,Eric Locke,Danny Stewart,0.0,1.0,815.0,1206.0,The First Old World Forgemaster,2024-02-03 14:00:00,...,2000,3,v1.1,N,N,12,Y,NaN,NaN,Ogre Kingdoms


So we currently have 2020 entries and 31 variables. We are not interested in all of these variables, I'll primarily be looking at player1_result, player2_result, player1_faction, player2_faction, points, and ruleset. The dataset also has two columns, full_data and mirror_match, that will help us filter for the data we want. The former identifies any match where one or both of the players have missing data (name, result, faction) and the latter is if the same faction plays against itself which will be excluded from analysis.

In [6]:
tow = tow.query("`mirror_match` == 'N' & full_data == 'Y'")
tow.shape

(1616, 31)

With the initial filtering done, we stand at 1616 samples. However, each sample contains the data for both players. To be able to do any analysis, we need to transform the dataset so each sample pertains to a single player's (or for our focus faction's) result.

In [7]:
player1_db = tow[['player1_faction', 'player2_faction' ,'player1_result', 'points', 'rounds', 'ruleset', 'players']] .rename(columns={"player1_faction": "player_faction", "player2_faction" : "opponent_faction", "player1_result" : "result"})
player2_db = tow[['player2_faction', 'player1_faction', 'player2_result', 'points', 'rounds', 'ruleset', 'players']].rename(columns={"player2_faction": "player_faction", "player1_faction" : "opponent_faction", "player2_result" : "result"})
player_db = pd.concat([player1_db, player2_db], ignore_index=True)
display(player_db)

,player_faction,opponent_faction,result,points,rounds,ruleset,players
0,W_Chaos,Wood Elves,0.0,2000,3,v1.1,12
1,Tomb Kings,High Elves,1.0,2000,3,v1.1,12
2,Bretonnia,Dwarves,0.0,2000,3,v1.1,12
3,Ogres,Dwarves,1.0,2000,3,v1.1,12
4,Wood Elves,Tomb Kings,1.0,2000,3,v1.1,12
...,...,...,...,...,...,...,...
3227,High Elves,Dark Elves,1.0,1250,3,v1.2,10
3228,Bretonnia,Skaven,0.0,1250,3,v1.2,10
3229,Tomb Kings,Ogres,1.0,1250,3,v1.2,10
3230,O&G,D_Chaos,0.0,1250,3,v1.2,10


There we go, 3232 samples with 7 variables. The dataset is now ready for us to start diving into.

## Analysis

We will start with establishing a baseline from where we can compare the rest of the analyses. So we will look at the win rates for all factions regardless of point size, tournament length, etc. Also, ties will be counted as half a win.

In [11]:
tow_faction_wr = player_db.groupby(['player_faction'])['opponent_faction'].count().reset_index()
wins_db = player_db.query("result != 0.0").groupby(['player_faction'])['result'].sum().reset_index()
tow_faction_wr['wins'] = wins_db['result']
tow_faction_wr.rename(columns={'opponent_faction':'games_played'}, inplace=True)
tow_faction_wr['win_rate'] = tow_faction_wr['wins'] / tow_faction_wr['games_played']
tow_faction_wr

,player_faction,games_played,wins,win_rate
0,Beastmen Brayherds,163,85.0,0.521472
1,Bretonnia,378,206.0,0.544974
2,Chaos Dwarfs,59,29.0,0.491525
3,D_Chaos,89,42.0,0.471910
4,Dark Elves,137,63.0,0.459854
5,Dwarves,258,115.5,0.447674
6,Empire,240,92.0,0.383333
7,High Elves,222,120.0,0.540541
8,Lizardmen,84,37.5,0.446429
9,O&G,320,164.0,0.512500


So what do we see here: four factions have less than one hundred games played and every faction has at least twenty-nine wins. Four factions have over three hundred games played. Of the sixteen factions, five have win rates outside the accepted "sweet spot" of 45-55%, all of which are under 45%.

Now that the baseline has been established, we can start to dive deeper into the data to see what we can find. The first dive is going to look at win rates when point sizes are accounted for. The first thing we do will determine the three most popular points sizes.

In [27]:
player_db.groupby(['points'])['player_faction'].count().sort_values(ascending=False).reset_index()

,points,player_faction
0,2000,1672
1,1500,836
2,1250,334
3,1999,128
4,1000,110
5,1750,106
6,2500,30
7,2250,10
8,500,6


In [ ]:
tk_db = player_db.query("player_faction == 'Tomb Kings of Khemri'").groupby(['opponent_faction'])['player_faction'].count()
tk_wins = player_db.query("player_faction == 'Tomb Kings of Khemri' & result == 2.0").groupby('opponent_faction')['result'].count()
tk_db = pd.concat([tk_db, tk_wins], axis=1).rename(columns={"player_faction": "games_played", "result": "wins"})
tk_db['win_rate_by_faction'] = tk_db['wins']/tk_db['games_played']
tk_db = tk_db.reset_index()
tk_db = tk_db.fillna(0)
tk_db

In [ ]:
tk_db['games_played'].sum()

In [ ]:
pn.ggplot(tk_db) + pn.aes(x='opponent_faction', y='win_rate_by_faction') + pn.geom_bar(stat = 'identity') + pn.geom_abline(intercept=tk_db['win_rate_by_faction'].mean(), slope=0, color="red") + pn.theme_bw() + pn.theme(axis_text_x = pn.element_text(angle=45, hjust=1)) + pn.labs(title="Tomb Kings Win Rate by Faction", subtitle='2000 points, >= 10 players, no mirror-matches', x="Opponent Faction", y='Win Rate')



In [ ]:
tk_db['win_rate_by_faction'].mean()